In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy.linalg as nla
import seaborn as sns
import scipy as sp
import pandas as pd
import multitype_new as mt
import importlib
importlib.reload(mt)
from scipy.integrate import odeint
from joblib import Parallel, delayed
from time import time

# Parameters to change 
ntypes = 8
# Number of types-at-birth
nexposed = int(ntypes/2)


# Disease Parameters

gamma = 0.25 # Recovery Rate
sigma = 1/3  # Progression rate from exposed to infectious

beta_baseline = 0.675

print('Running for beta = ' + str(beta_baseline))

const_vec = np.squeeze(pd.read_csv('const_vec_in.csv',header=None).to_numpy()) #np.array((1., sus_ur, sus_vu, sus_vr))
prop_vec = np.squeeze(pd.read_csv('prop_vec_in.csv',header=None).to_numpy()) #np.array((p_sus*p_no_vac, p_rec*p_no_vac, p_sus*p_vac, p_rec*p_vac))

scale = const_vec*prop_vec

epsilon = 1e-3
RV = beta_baseline/gamma

Tmax = 400 
num = Tmax 
time_vec = np.linspace(0, Tmax, num=num)
dtime = time_vec[1]-time_vec[0]


In [ ]:
im = 0
im_vec = np.zeros(ntypes)
y0 = np.zeros(ntypes)
y0[0] = 1
ics = np.zeros(ntypes)
def eta(t):
    return im_vec

trans_vec = np.ones(nexposed)

def lifetime_vec(beta, gamma, sigma, ntypes = ntypes, const_vec = const_vec, prop_vec = prop_vec, trans_vec = trans_vec):
    nexposed = int(ntypes / 2)
    omega_vec = sigma * np.ones(ntypes) 

        
    for nex in range(0, nexposed):
        omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta)*trans_vec[nex] + gamma
    return omega_vec

omega_vec = lifetime_vec(beta_baseline, gamma, sigma)

def P(u, t): # Offspring distribution

        ## Returns a vector of length ntypes with entry i containing  probabilities of generating particles 
        ## of each type from a particle of type i

        pvec = np.zeros_like(u)
        nexposed = int(ntypes/2)
        omega = (np.sum(prop_vec * const_vec * beta_baseline) + gamma)
        

        sum_gen_funcs = np.sum((beta_baseline*prop_vec/omega)*const_vec*u[:nexposed])
         
        for nex in range(0, nexposed):

            pvec[nex] = u[nex+nexposed] 
            pvec[nex+nexposed] =  u[nex+nexposed] * sum_gen_funcs  + (gamma/omega)
        
        return pvec  

def Jacobian_mat(beta, gamma, sigma, ntypes = ntypes, const_vec = const_vec, prop_vec = prop_vec, trans_vec = trans_vec, omega_vec = omega_vec):
    
    # Jacobian of the offspring distribution - corresponding to \Omega

    Omat = np.zeros((ntypes, ntypes))
    omega_vec = sigma * np.ones(ntypes)       
    nexposed = int(ntypes/2)
    for nex in range(0, nexposed):
        omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta)*trans_vec[nex] + gamma
        Omat[nex, nex+nexposed] = omega_vec[nex]
        Omat[nex+nexposed, :nexposed] = const_vec*prop_vec*beta *trans_vec[nex]
        Omat[nex+nexposed, nex+nexposed] = np.sum(const_vec * beta * prop_vec)*trans_vec[nex]

    Omat -= np.diag(omega_vec)
    return Omat
	
	
rho = (odeint(mt.set_odes, ics, time_vec, args = (P, omega_vec)).T )
q = np.ones_like(time_vec)
for i in range(ntypes):
    q *= rho[i, :] ** y0[i]

np.savetxt('./Outputs_for_matlab/p_extinction/p_extinction_R=' + str(RV) + '.csv', np.vstack((time_vec, q)))

# Jacobian, or mean matrix
Omat = Jacobian_mat(beta_baseline, gamma, sigma)
# Mean of each type
Mmat = odeint(mt.set_mean_odes, y0, time_vec, args = (Omat,)).T
# Total mean
mean = np.sum(Mmat, axis=0)

variance = mt.variance(time_vec, y0, omega_vec, Omat, [prop_vec, const_vec, sigma, 
                                                   beta_baseline, gamma], eta)

#Coefficient of variation
sig_over_mean = (np.sqrt(variance)/(mean))
	
Tstar_idx, Tstar = mt.Tstar(time_vec, q, sig_over_mean, thresh1 = epsilon, thresh2 = epsilon)
Zstar_min= mean[Tstar_idx]

eigvls, orth = nla.eig(Omat)
orth = np.real_if_close(orth)
if True:#np.iscomplex(orth).any():

# Maybe change this back
    print('Numerical instability: Switching to symbolic compution of eigenvectors')
    sym_params = [scale[0], scale[1], scale[2], scale[3], sigma, gamma, beta_baseline]
    sym_labels = ['a', 'b', 'c', 'd', 's', 'g', 'beta']
    sym_dict = dict([(l, p) for (l, p) in zip(*(sym_labels, sym_params))])
    diag_sym = mt.Omega_sym(0).diagonalize(normalize = True)
    sym_eigvectors = diag_sym[0]
    sym_eigvalues = diag_sym[1]
    eigvls = np.diag(np.array(sym_eigvalues.subs(sym_dict)))
    orth = np.array(sym_eigvectors.subs(sym_dict)).astype(float)
assert(not np.iscomplex(orth).any()), 'Jacobian has complex eigendecomposition'
eigvls = eigvls.astype('float64')
orth = orth.astype('float64')

# Some issue with ordering in sympy vs numpy
ordering = np.flip(np.argsort(eigvls)).tolist()
eigvls, orth = mt.reorder_evecs(eigvls, orth, ordering)


diagmat = np.diag(eigvls)
growth_rate = np.max(eigvls)
assert(growth_rate > 0), 'Growth Rate must be positive (i.e. Branching Process must be supercritical)'

orth_inv = mt.invert_mat_safe(orth.copy())
orth_c = orth.copy().T
orthc_inv = mt.invert_mat_safe(orth_c.copy())


### Build variance matrix via Kronecker Products
H = np.kron(orth, np.kron((orthc_inv), orthc_inv))
Hinv = np.kron(orth_inv, np.kron(orth_c, orth_c)) 
Amat = np.kron(orthc_inv, orthc_inv)
Amat_inv = np.kron(orth_c, orth_c)

vec_w = np.zeros(ntypes**3)

# Build Hessian matrix
Hessian_mat = np.zeros((ntypes, ntypes, ntypes))
for level in range(0, nexposed):
    for i in range(0, nexposed):

        Hessian_mat[level + nexposed, level+nexposed, i] = (beta_baseline * const_vec[i]*prop_vec[i])/(omega_vec[level+nexposed])
        Hessian_mat[level + nexposed, i, level+nexposed] = (beta_baseline * const_vec[i]*prop_vec[i])/(omega_vec[level+nexposed])

P_mat = np.zeros((ntypes, ntypes))


for nex in range(0, nexposed):
    P_mat[nex, nex+nexposed] = omega_vec[nex]/omega_vec[nex]
    P_mat[nex+nexposed, :nexposed] = const_vec*prop_vec*beta_baseline/omega_vec[nex+nexposed]
    P_mat[nex+nexposed, nex+nexposed] = np.sum(const_vec * beta_baseline * prop_vec)/omega_vec[nex+nexposed]

P_mat = P_mat.T

Gmat = np.zeros((ntypes, ntypes, ntypes))
C = np.zeros(ntypes**3)
for l in range(0, ntypes):
    f_vec = (P_mat - np.identity(ntypes))[l, :]
    unitvec = np.zeros(ntypes)
    unitvec[l] = 1

    Gmat[l, :, :] = Hessian_mat[l, :, :] - np.diag(P_mat[:, l]) 
    Gmat[l, :, :] += np.diag(unitvec)        
    Gmat[l, :, :] *= omega_vec[l]


growth_rate = np.max(eigvls)
print(growth_rate)

escale = (Mmat[:, Tstar_idx] / orth[:, 0])[0]
integral_limit = np.sum(Mmat[:, Tstar_idx]/np.sum(orth[:, 0]))


Sigma_mat = Jacobian_mat(beta_baseline, gamma = gamma, sigma = sigma)
Sigma_mat[nexposed:, :nexposed] = 0
T_mat = Jacobian_mat(beta_baseline, gamma = gamma, sigma = sigma) - Sigma_mat
next_gen = - T_mat @ nla.inv(Sigma_mat)
R_effective = np.max(nla.eigvals(next_gen))
print('R_eff = ' + str(R_effective))


In [236]:
num_sims = 10000

class Gillespie:
    def __init__(self, rates, transitions, y0):
        
        assert(len(rates(y0)) == len(transitions))
        
        self.rates = rates
        self.y0 = y0
        self.numvars = len(y0)
        self.transitions = transitions
        return None
    
    
    def Gillespie_trajectory(self, Zstar = None, Tstar = None, max_events = int(1e7)):
        rates = self.rates
        y0 = self.y0
        numvars = self.numvars
        transitions = self.transitions
        output_mat = np.zeros((numvars+1, max_events))
        output_mat[1:, 0] = y0
        rands = np.random.random(size = (2, max_events))
        t = 0
        hitting_time = None
        case_check = 0
        for i in range(1, max_events):
            
            # Calculate rates
            
            rate_vec = rates(output_mat[1:, i-1])
            rate_cumsum = np.cumsum(rate_vec)
            rate_sum = np.sum(rate_vec)
            dt = -np.log(rands[0, i-1])/rate_sum
            
            # Update
            
            t += dt
            output_mat[0, i] = t
            choose_rand_event = rate_sum * rands[1, i-1]
            event_index = np.where(rate_cumsum > choose_rand_event)[0][0]
            
            output_mat[1:, i] = output_mat[1:, i-1] + transitions[event_index]

            total_infectious = np.sum(output_mat[1:, i])
            if Zstar is not None:
                if isinstance(Zstar, int) or isinstance(Zstar, float):
                    if total_infectious > Zstar:

                        hitting_time = t

                        return hitting_time

            
            if Tstar is not None:
                if t > Tstar:
                    check_cases = output_mat[1:, i]
                    return check_cases
                    
            if total_infectious == 0:
                break
            
        if Zstar is not None:
            return None
        elif Tstar is not None:
            return None
        else:
            return output_mat
        
def do_Gillespie_runs(G, Zstar):
    result = None
    while result == None:
        result = G.Gillespie_trajectory(Zstar = integral_limit)
    return result

In [237]:
# Choose index of Istars_min (Z^*) to obtain FPT distribution - improve by doing this for all!! 
im_gillespie = im

def rates(u):           
    rates_vec = np.zeros(3*nexposed)
    rates_vec[:nexposed] = sigma*np.ones(nexposed)*u[:nexposed]
    rates_vec[nexposed:(2*nexposed)] = np.sum(const_vec*prop_vec*beta_baseline*u[nexposed:]) #+ im_gillespie*prop_vec*const_vec
    rates_vec[(2*nexposed):(3*nexposed)] = gamma*u[nexposed:]
    return rates_vec

transitions = {}
for i in range(nexposed):
    exposed_to_infectious_trans = np.zeros(ntypes) #E_i -> I_i
    exposed_to_infectious_trans[i] = -1
    exposed_to_infectious_trans[i+nexposed] = 1

    transitions[i] = exposed_to_infectious_trans

    infection_trans = np.zeros(ntypes)
    infection_trans[i] = 1

    transitions[i+nexposed] = infection_trans

    recovery_trans = np.zeros(ntypes)
    recovery_trans[i+nexposed] = -1

    transitions[i + 2*nexposed] = recovery_trans


y0vec = np.zeros(ntypes)
if im_gillespie == 0:
    y0vec[0] = 1
G = Gillespie(rates, transitions, y0vec)
# Reff_cols = [str(Re) for Re in Reff_vec]

In [ ]:
n_cores = 6
start = time()
some_runs = Parallel(n_jobs=n_cores)(delayed(do_Gillespie_runs)(G, integral_limit) for m in range(num_sims))
end = time()
print('Parallel Gillespie simulations on ' + str(n_cores) + ' cores finished in ' + str(np.round(end - start, 2)) + ' seconds!')
sns.kdeplot(some_runs)

In [ ]:
# Read in simulation output if not wanting to rerun 
some_runs = np.genfromtxt("./Plotting_Outputs/gillespie_sims_beta=" + str(beta_baseline) + ".csv", delimiter=",")
print('Reading simulation output for beta = ' + str(beta_baseline))

In [ ]:
if (orth[:, 0] < 0).all():
    orth = -orth
change_from_ebasis = orth 

evec = change_from_ebasis[:, 0] 

variance_diffusion_mat = np.zeros((ntypes, ntypes))

for l in range(ntypes): 
    variance_diffusion_mat += Gmat[l, :, :] *evec[l]  
variance_all = (orth[:, 0].T@variance_diffusion_mat@orth[:, 0])

print('Total variance is ' + str(variance_all))

evec_scaling = np.abs(1/np.sum(evec))

print('Evec scaling is ' + str(evec_scaling))

xvec = np.linspace(np.finfo(float).eps, 20000, int(1e5))
xvec_idx = np.min(np.where(xvec>=(evec_scaling*Zstar_min))[0])
integral_limit = xvec[xvec_idx]

t0 = 0
dt = np.diff(time_vec)[0]
cmp = sns.color_palette('Set2')

stop_cases = [integral_limit]
cols = [str(st) for st in stop_cases]


def chi_sq(u, x, t):
    x_scale = 2*growth_rate*x/(((variance_all/2))*(np.exp(growth_rate*t) - 1))
    lamb = 2*growth_rate*np.exp(growth_rate*t)/((variance_all/2)*(np.exp(growth_rate*t) - 1))
    chi_sq_pdf = growth_rate/((variance_all/2)*(np.exp(growth_rate*t) - 1)) * np.sqrt(np.exp(growth_rate*t)/x) * np.exp(- 1/2 * (lamb + x_scale)) * sp.special.iv(1, np.sqrt(x_scale*lamb)) / ((1-np.exp(-lamb/2)))
    return chi_sq_pdf


print('Critical V* = ' +str(xvec[xvec_idx]) + ', critical Z^* is ' + str(Zstar_min))
cdf_chisq = np.zeros(len(time_vec))

for T_idx, T_val in enumerate(tqdm(time_vec)):
    chi_sq_integral = sp.integrate.odeint(chi_sq, 0, xvec, args = (T_val,)).flatten() 
    prob =  (1-(chi_sq_integral[xvec_idx]))
    cdf_chisq[T_idx] = prob
    
pdf_chisq = np.gradient(cdf_chisq, dt)
plt.plot(time_vec, pdf_chisq, label = "Feller_exact", color = cmp[1])
sns.histplot(some_runs, label = 'Simulations', stat = 'density', color = cmp[2], alpha = 0.5, bins = 40)
sns.kdeplot(some_runs, color = cmp[2])
plt.xlim([0, 300])
plt.legend()


In [240]:
np.savetxt("./Plotting_Outputs/gillespie_sims_beta=" + str(beta_baseline) + ".csv", np.array(some_runs), delimiter=",")
np.savetxt("./Plotting_Outputs/gillespie_pdf_beta=" + str(beta_baseline) + ".csv", pdf_chisq,  delimiter=",")